In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'Utils')
import ArtistExtractor as AE
from ast import literal_eval
import moreFunction

In [2]:
columns = ["place","src","address","location","canton","event","date","artists","genre","coordinates"]
a = pd.DataFrame(columns=[columns])
lineup1 = ["Joachim Garraud","**David Guetta**","Panteros666 ✱ ✲","Rihanna (Live)"]
lineup2 = ["02:20 - Rihanna", " Drum'n'bass", "David Guetta"]
a.loc[0] = ["test","club","address","location_test","NE","simsalabim","2002-22-02",str(lineup1),"Classique","(1.0,1.0)"]
a.loc[1] = ["test2","club","address","location_test","VD","simsalabim","2002-22-02",str(lineup2),"Pop","(1.0,1.0)"]
a

,place,src,address,location,canton,event,date,artists,genre,coordinates
0,test,club,address,location_test,NE,simsalabim,2002-22-02,"['Joachim Garraud', '**David Guetta**', 'Pante...",Classique,"(1.0,1.0)"
1,test2,club,address,location_test,VD,simsalabim,2002-22-02,"['02:20 - Rihanna', "" Drum'n'bass"", 'David Gue...",Pop,"(1.0,1.0)"


In [3]:
def getArtistsList(df):
    ArtistsDataFrame = pd.DataFrame()
    i=0
    for index,row in df.iterrows():
        lineup = row["artists"] #get line-up
        artists = literal_eval(lineup)
        for a in artists:
            a = a.split("(")[0]#Remove parenthesis at end of artist (record label, live act..)

            artist_list = []
            artist_list.append(a)
            cleaned_artist = moreFunction.clean_artists(artist_list)
            if(len(cleaned_artist)>0):
                a = cleaned_artist[0]
            a = pd.Series(a.strip())
            ArtistsDataFrame = ArtistsDataFrame.append(a,ignore_index=True)
        i+=1
        if(i%5000==0):
            print(i)

    ArtistsDataFrame.columns=["artist"]
    #ArtistsDataFrame.to_csv("ClubDataTest/ArtistsList.csv",encoding="utf-8")
    #ArtistsDataFrame.head(10)
    return ArtistsDataFrame.drop_duplicates()

In [4]:
artists = getArtistsList(a)

In [5]:
def downloadGenresSpotify(artistsDF,begin=0,end=100000):

    Artists = artistsDF.copy()
    Artists = Artists.drop_duplicates().reset_index()[["artist"]]
    Artists.columns=["artist"]
    Artists["genres_spotify"]=None
    Artists["genres_ra"]=None
    Artists["genres_wiki"]=None
    Artists["main_genres"]=None
    Artists["top3_genres"]=None
    Artists["genre"]=None    
    
    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres of "+str(end-begin)+" artist...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_spotify = AE.getGenresFromSpotify(artist)
        
        if(len(genres_spotify)<1):
            genres_spotify = None
        Artists.loc[Artists.index==i,"genres_spotify"] = str(genres_spotify)

        if(i%300==0):
            Artists.to_csv(path,encoding="utf-8")
            print(str(i)+": Saved under "+path)
    
    Artists.to_csv(path,encoding="utf-8")
    
    print(str(i)+": Saved under "+path)
    last_read = i

    print("Last read = "+str(last_read))
    return Artists

In [6]:
artists = downloadGenresSpotify(artists)
artists

0: Saved under FullData/ArtistDataframe_0_6.csv
5: Saved under FullData/ArtistDataframe_0_6.csv
Last read = 5


,artist,genres_spotify,genres_ra,genres_wiki,main_genres,top3_genres,genre
0,Joachim Garraud,"['big room', 'edm', 'electro house', 'progress...",None,None,None,None,None
1,David Guetta,"['dance pop', 'edm', 'pop', 'tropical house']",None,None,None,None,None
2,Panteros666 ✱ ✲,['destroy techno'],None,None,None,None,None
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None
4,- Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None
5,Drum'n'bass,None,None,None,None,None,None


In [7]:
def createDictionnaryFromArtists(artists):
    print("Creating dictionnary from wikipedia...")
    dic = AE.createDictionnary()
    print("Adding genres from artists (spotify genres)")
    genres_list = []
    for id,row in artists.iterrows():
        genres = literal_eval(row.genres_spotify)
        if(genres!=None):
            genres_list+=(genres)
        
    #print(genres_list)
    dic = AE.updateDictionnary(genres_list,dic)
    return dic                

In [8]:
def downloadGenresWikipediaAndRA(Artists,dictionnary,begin=0,end=100000):

    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    #path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres of "+str(end-begin)+" artist...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_wiki = AE.getGenresFromWikipedia(artist,dictionnary)
        genres_ra = AE.getGenresFromRA(artist,dictionnary)
        if(genres_wiki== None or len(genres_wiki)<1):
            genres_wiki = None
        Artists.loc[Artists.index==i,"genres_wiki"] = str(genres_wiki)
        Artists.loc[Artists.index==i,"genres_ra"] = str(genres_ra)

        if(i%300==0):
            #Artists.to_csv(path,encoding="utf-8")
            print(str(i+1))
    
    #Artists.to_csv(path,encoding="utf-8")
    
    #print(str(i)+": Saved under "+path)
    last_read = i

    print("Last read = "+str(last_read+1))
    return Artists

In [9]:
dic = createDictionnaryFromArtists(artists)
artists = downloadGenresWikipediaAndRA(artists,dic)
artists


Creating dictionnary from wikipedia...
Adding genres from artists (spotify genres)
1
Last read = 6


,artist,genres_spotify,genres_ra,genres_wiki,main_genres,top3_genres,genre
0,Joachim Garraud,"['big room', 'edm', 'electro house', 'progress...",None,"['complextro', 'techno', 'electronic', 'edm', ...",None,None,None
1,David Guetta,"['dance pop', 'edm', 'pop', 'tropical house']","['techno', 'rock', 'electropop', 'coldwave', '...","['pop', 'electropop', 'chanson', 'funk', 'rap'...",None,None,None
2,Panteros666 ✱ ✲,['destroy techno'],None,None,None,None,None
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,"['drill', 'pop', 'chanson', 'country', 'soul',...",None,None,None
4,- Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None
5,Drum'n'bass,None,None,"['drill', 'step', 'raggacore', 'techstep', 'gr...",None,None,None


In [10]:
def extendingGenres(artists_df,dictionnaryOfGenres,debug=False):
    genres = list(set(dictionnaryOfGenres.values()))
    genres.append("unknown")
    
    #Extending
    #Getting columns
    columnsArtists = []
    for c in artists_df.columns:
        if("Unnamed" not in c):
            columnsArtists.append(c)
    columnsArtists = columnsArtists+genres+["total_genres"]

    print("Extending columns to these genres :")
    print(columnsArtists)
    
    i=0
    for id,row in artists_df.iterrows():
        i+=1
        if(debug and i%3000==0):
            print(i)
        main_genres = row["main_genres"]
        
        
        total = 0.0
        
        if(main_genres!=None and main_genres!="None"):
            total = len(literal_eval(main_genres))
            
            #Check if artist has a genre (from other dataframes)
            if(total==0):
                if(raw["genre"]!=None and raw["genre"]!="None"):
                    main_genres = list(literal_eval(raw["genre"]))
                    artists_df.loc[id,"main_genres"] = main_genres
                    total = 1.0
                
        artists_df.loc[id,"total_genres"]=total
        
        #percententage of this genre
        for c in genres:
            if(total==0):
                artists_df.loc[id,c] = 0.0
                artists_df.loc[id,"unknown"]=1.0
            else:
                artists_df.loc[id,c] = main_genres.count(c)/total

    return artists_df

In [11]:
artists = extendingGenres(artists,dic)

Extending columns to these genres :
['artist', 'genres_spotify', 'genres_ra', 'genres_wiki', 'main_genres', 'top3_genres', 'genre', 'folk', 'blues', 'pop', 'caribbean and caribbean-influenced', 'latin', 'orchestral', 'jazz', 'country', 'electronic', 'r&b and soul', 'african', 'comedy', 'avant-garde', 'hip hop', 'asian', 'rock', 'unknown', 'easy listening', 'unknown', 'total_genres']


In [12]:
artists

,artist,genres_spotify,genres_ra,genres_wiki,main_genres,top3_genres,genre,total_genres,folk,unknown,...,country,electronic,r&b and soul,african,comedy,avant-garde,hip hop,asian,rock,easy listening
0,Joachim Garraud,"['big room', 'edm', 'electro house', 'progress...",None,"['complextro', 'techno', 'electronic', 'edm', ...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,David Guetta,"['dance pop', 'edm', 'pop', 'tropical house']","['techno', 'rock', 'electropop', 'coldwave', '...","['pop', 'electropop', 'chanson', 'funk', 'rap'...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Panteros666 ✱ ✲,['destroy techno'],None,None,None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,"['drill', 'pop', 'chanson', 'country', 'soul',...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,- Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Drum'n'bass,None,None,"['drill', 'step', 'raggacore', 'techstep', 'gr...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
def extendsMainGenres(artists,dictionnary):
    
    
    artists["all_genres"] = None
    #dictionnaryOfGenres = LU.loadDictionary("FullData/AllGenresDic",enc="utf-8")
    i=0
    for id,row in artists.iterrows():
        i+=1
        #if(i>20):
         #   break
        if(i%1000==0):
            print(i)

        artist = row["artist"]
        genres_ra = row.genres_ra
        genres_s = row.genres_spotify
        genres_w = row.genres_wiki
        #genres_s =str(None)
        
        print(genres_ra)
        print(genres_s)
        print(genres_w)
        
        print(".....")
        '''
        if(len(gs)>0):
            genres_s = ["genres_spotify"].values[0]

        #genres_a = str_df2.replace("{","[").replace("}","]")

        if(genres_s!="None"):
            genres_s = literal_eval(genres_s)
        else:
            genres_s = []

        if(genres_a!="None" and str(genres_a)!="nan"):
            try:
                genres_a = literal_eval(genres_a)
            except:
                print(genres_a)
        else:
            genres_a = []
            '''

        genres = genres_s+genres_ra+genres_w
        print(genres)
        print(":..................")

        main_genres = AE.mainGenres(genres,dictionnary)
        top3 = AE.getMaxGenre(main_genres,3)
        main_genre = None

        if(top3!=None and len(top3)>1):
            main_genre = top3[0]


        if(len(genres)==0):
            genres = None
        if(len(genres_ra)==0):
            genres_a = None
        if(len(main_genres)==0):
            main_genres = None

        artists.loc[id,"genres_ra"] = str(genres_ra)
        artists.loc[id,"main_genres"] = str(main_genres)
        artists.loc[id,"top3_genres"] = str(top3)
        artists.loc[id,"all_genres"] = str(genres)
        artists.loc[id,"genre"] = main_genre


    #df3.to_csv("FullData/ArtistDF_withGenres.csv",encoding="utf-8")
    return artists

In [35]:
extendsMainGenres(artists,dic)

None
['big room', 'edm', 'electro house', 'progressive electro house', 'progressive house']
['complextro', 'techno', 'electronic', 'edm', 'pop']
.....
['big room', 'edm', 'electro house', 'progressive electro house', 'progressive house']None['complextro', 'techno', 'electronic', 'edm', 'pop']
:..................
['techno', 'rock', 'electropop', 'coldwave', 'pop', 'rap', 'classic']
['dance pop', 'edm', 'pop', 'tropical house']
['pop', 'electropop', 'chanson', 'funk', 'rap', 'rock', 'edm', 'son']
.....
['dance pop', 'edm', 'pop', 'tropical house']['techno', 'rock', 'electropop', 'coldwave', 'pop', 'rap', 'classic']['pop', 'electropop', 'chanson', 'funk', 'rap', 'rock', 'edm', 'son']
:..................
None
['destroy techno']
None
.....
['destroy techno']NoneNone
:..................
None
['dance pop', 'pop', 'pop christmas', 'r&b', 'soul christmas', 'urban contemporary']
['drill', 'pop', 'chanson', 'country', 'soul', 'dubstep', 'rap', 'eurodance', 'rock', 'son']
.....
['dance pop', 'pop'

,artist,genres_spotify,genres_ra,genres_wiki,main_genres,top3_genres,genre,total_genres,folk,unknown,...,electronic,r&b and soul,african,comedy,avant-garde,hip hop,asian,rock,easy listening,all_genres
0,Joachim Garraud,"['big room', 'edm', 'electro house', 'progress...",None,"['complextro', 'techno', 'electronic', 'edm', ...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['big room', 'edm', 'electro house', 'progress..."
1,David Guetta,"['dance pop', 'edm', 'pop', 'tropical house']","['techno', 'rock', 'electropop', 'coldwave', '...","['pop', 'electropop', 'chanson', 'funk', 'rap'...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['dance pop', 'edm', 'pop', 'tropical house'][..."
2,Panteros666 ✱ ✲,['destroy techno'],None,None,None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['destroy techno']NoneNone
3,Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,"['drill', 'pop', 'chanson', 'country', 'soul',...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['dance pop', 'pop', 'pop christmas', 'r&b', '..."
4,- Rihanna,"['dance pop', 'pop', 'pop christmas', 'r&b', '...",None,None,None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['dance pop', 'pop', 'pop christmas', 'r&b', '..."
5,Drum'n'bass,None,None,"['drill', 'step', 'raggacore', 'techstep', 'gr...",None,None,None,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"NoneNone['drill', 'step', 'raggacore', 'techst..."
